# Tim Gormly
## Module 5, Task 4
### 2/13/2023

In this notebook, we will use SQL to perform CRUD operations on a sample database.

<hr>

### Create a connection to the database:

In [517]:

import sqlite3

connection = sqlite3.connect(r'C:\Users\timgo\OneDrive\Desktop\School\Spring2023\IntroToPython-master\IntroToPython-master\examples\IntroToPython\examples\ch17\sql\books.db')

<hr>

### Updating Pandas display options

We will update pandas so that only 10 columns are displayed.

In [518]:
import pandas as pd

pd.options.display.max_columns = 10

<hr>

### Use pd read_sql to select data from tables

Let's take a look at the data stored in the three tables in the books database.

In [519]:
pd.read_sql('SELECT * FROM authors', connection)

,id,first,last
0,1,Paul,Deitel
1,2,Harvey,Deitel
2,3,Abbey,Deitel
3,4,Dan,Quirk
4,5,Alexander,Wald


Note that the Pandas DataFrame adds its own index to the leftmost column of our table.  We can adjust this by adding a custom index to our DataFrame, within the read_sql function.

In [520]:
pd.read_sql('SELECT * FROM titles', connection, index_col=['isbn'])

,title,edition,copyright
isbn,,,
0135404673,Intro to Python for CS and DS,1,2020
0132151006,Internet & WWW How to Program,5,2012
0134743350,Java How to Program,11,2018
0133976890,C How to Program,8,2016
0133406954,Visual Basic 2012 How to Program,6,2014
0134601548,Visual C# How to Program,6,2017
0136151574,Visual C++ How to Program,2,2008
0134448235,C++ How to Program,10,2017
0134444302,Android How to Program,3,2017


In [521]:
pd.read_sql('SELECT * FROM author_ISBN', connection, index_col=['id'])

,isbn
id,
1,0134289366
2,0134289366
5,0134289366
1,0135404673
2,0135404673
1,0132151006
2,0132151006
3,0132151006
1,0134743350


<hr>

## 1. SELECT

In the previous examples, we used * to request all data within the table.  Instead, we can specify which specific columns we would like to receive results for in the SELECT clause.  In this example, we will select only the first and last names from the authors table.

In [522]:
pd.read_sql('SELECT first, last FROM authors', connection)

,first,last
0,Paul,Deitel
1,Harvey,Deitel
2,Abbey,Deitel
3,Dan,Quirk
4,Alexander,Wald


<hr>

## 2. WHERE

The WHERE clause allows us to add criteria to our selection.  Perhaps we want to see only books copywritten after 2016.

In [523]:
pd.read_sql(""" SELECT title, edition, copyright 
                FROM titles 
                WHERE copyright > '2016'""", connection)

,title,edition,copyright
0,Intro to Python for CS and DS,1,2020
1,Java How to Program,11,2018
2,Visual C# How to Program,6,2017
3,C++ How to Program,10,2017
4,Android How to Program,3,2017


With the WHERE clause, we only received 5 results.  Without it, we would have pulled all 10 rows in the titles table.

We can use SQL wildcards in the WHERE clause to find results that match specific patterns.  For example, we can use the % wildcard to represent zero or more characters in the position of the %.  All author's with a last name starting with D can be found in this way:

In [524]:
pd.read_sql(""" SELECT id, first, last 
                FROM authors 
                WHERE last LIKE 'D%'""", connection, index_col=['id'])

,first,last
id,,
1,Paul,Deitel
2,Harvey,Deitel
3,Abbey,Deitel


The _ wildcard can be used to represent exactly 1 character in a given space.  We could find authors who's first name begins with any character, has a second character of 'b', and has zero or more characters after that 'b' using the following query:

In [525]:
pd.read_sql(""" SELECT id, first, last 
                FROM authors 
                WHERE first LIKE '_b%'""", connection, index_col=['id'])

,first,last
id,,
3,Abbey,Deitel


<hr>

## 3. ORDER BY 

The ORDER BY clause allows us to specify how the rows returned by a query should be ordered.  By default, results specified in the ORDER BY clause will be resulted ascending (A to Z, 1 to 100, etc).  You can also explicitly state that your results should be in ascending order if you choose.  In the following example, we will SELECT items from the titles table, and sort them in ascending order based on the title of the book.

In [526]:
pd.read_sql(""" SELECT title 
                FROM titles 
                ORDER BY title ASC""", connection)

,title
0,Android 6 for Programmers
1,Android How to Program
2,C How to Program
3,C++ How to Program
4,Internet & WWW How to Program
5,Intro to Python for CS and DS
6,Java How to Program
7,Visual Basic 2012 How to Program
8,Visual C# How to Program
9,Visual C++ How to Program


We'll run the query once more without ASC included.  Note that the results are the same.

In [527]:
pd.read_sql(""" SELECT title 
                FROM titles 
                ORDER BY title""", connection)

,title
0,Android 6 for Programmers
1,Android How to Program
2,C How to Program
3,C++ How to Program
4,Internet & WWW How to Program
5,Intro to Python for CS and DS
6,Java How to Program
7,Visual Basic 2012 How to Program
8,Visual C# How to Program
9,Visual C++ How to Program


In the examples above, we sorted our result table by only one column.  The ORDER BY clause accepts a comme-separated list of values to sort by.  The first value in the list determines initial sorting.  Where first values are equal, the second value determines the order of items with identical first values, and so on.

In [528]:
pd.read_sql(""" SELECT id, first, last
                FROM authors
                ORDER BY last, first""", connection)

,id,first,last
0,3,Abbey,Deitel
1,2,Harvey,Deitel
2,1,Paul,Deitel
3,4,Dan,Quirk
4,5,Alexander,Wald


The result table is sorted so that the rows are in alphabetical order by last name.  Where the last name is identical among multiple rows, those rows are sorted in alphabetical order by first name.

By placing DESC after criteria, we can indicate that sorting should be handled descending, or in reverse order (Z to A, 100 to 0, etc.).  In the following example, we will sort by last name, <em>descending</em>, and first name, <em>ascending</em> 

In [529]:
pd.read_sql(""" SELECT id, first, last
                FROM authors
                ORDER BY last DESC, first""", connection, index_col=['id'])

,first,last
id,,
5,Alexander,Wald
4,Dan,Quirk
3,Abbey,Deitel
2,Harvey,Deitel
1,Paul,Deitel


The ORDER BY clause can be used in conjunction with the WHERE clause.  In SQL syntax, the ORDER BY clause comes after the WHERE clause.

In [530]:
pd.read_sql(""" SELECT isbn, title, edition, copyright
                FROM titles
                WHERE title LIKE '%How to Program'
                ORDER BY title""", connection)

,isbn,title,edition,copyright
0,0134444302,Android How to Program,3,2017
1,0133976890,C How to Program,8,2016
2,0134448235,C++ How to Program,10,2017
3,0132151006,Internet & WWW How to Program,5,2012
4,0134743350,Java How to Program,11,2018
5,0133406954,Visual Basic 2012 How to Program,6,2014
6,0134601548,Visual C# How to Program,6,2017
7,0136151574,Visual C++ How to Program,2,2008


The previous query uses a wildcard in its WHERE clause to return rows where the values in the title column end with 'How to Program'.  These rows are then ordered alphabetically by their title.

<hr>

## 4. INNER JOIN

Suppose you need your query to return data from multiple tables in its result table.  This can be accomplished using JOIN.  Specifically, we will look at INNER JOIN, which is similar to a Set Intersection in Python.

The following query will use the authors & authors_ISBN tables to produce a table with one row for every row in the two tables where the values held in the id columns in the two fields match.  When working with multiple tables, dot notation is used to specify columns.

In [531]:
pd.read_sql(""" SELECT first, last, isbn
                FROM authors
                INNER JOIN author_ISBN 
                    ON authors.id = author_ISBN.id
                ORDER BY last, first""", connection).head()

,first,last,isbn
0,Abbey,Deitel,0132151006
1,Abbey,Deitel,0133406954
2,Harvey,Deitel,0134289366
3,Harvey,Deitel,0135404673
4,Harvey,Deitel,0132151006


The logic in the ON statement works similarly to the WHERE clause.  You match values known as primary keys and foreign keys:
<ul>
    <li>Primary Key - a column that must have unique, non-null, values in the table.
        In the above query, author_ISBN is the primary key</li> 
    <li>Foreign Key - a column within a table that is meant to match the primary key in another table.  In the above query, id (from the authors table) is the foreign key that connects the authors table to the author_ISBN table.</li> 
</ul>
<hr>

## 5. INSERT INTO

So far, we have retrieved data from the database.  We can also <strong>modify</strong> the database.  Do do this, we will need a Cursor object from the sqlite3 module.  Our connection object informs its cursor method what database it needs to connect to.

In [532]:
cursor = connection.cursor()

We have created a variable named cursor, and assigned the result of sqlite3's cursor() method to it.  When a cursor is created, it receives a SQL query, in string format, as an argument.  The cursor then executes this SQL query when the cursor object is created.

In [533]:
cursor = cursor.execute(""" INSERT INTO authors (first, last)
                            VALUES ('Sue', 'Red')""")

The query above inserts a new row into the authors table (as in INSERT INTO authors) with values provided for (first, last).  VALUES then declares the values for the new row to be 'Sue', 'Red'.  These are assigned the the first and last columns specified earlier in the query.

In [534]:
pd.read_sql('SELECT id, first, last FROM authors', connection, index_col=['id'])

,first,last
id,,
1,Paul,Deitel
2,Harvey,Deitel
3,Abbey,Deitel
4,Dan,Quirk
5,Alexander,Wald
6,Sue,Red


A 6th row has been added to the authors table, Sue Red.

<hr>

## 6. UPDATE

Similar to the INSERT INTO statement, the UPDATE statement can be used to modify values in existing rows.  This statement also requires a cursor object.  The syntax is similar to what we've seen before.

In [535]:
cursor = cursor.execute(""" UPDATE authors SET last='Black'
                            WHERE last='Red' AND first='Sue'""")

The table listed after UPDATE is the table where the update will occur.  SET specifies the value(s) that are to be updated and what they are to be updated to: column='New Value'.  WHERE functions as it has in the past. In this case, WHERE filters the table to select the rows where the update should occur, instead of selecting the rows that are to be returned in a SELECT statement.

The cursor has a rowcount attribute.  This contains an integer value that represents the number of rows that were modified by its query.  cursor.rowcount() returning 0 indicates that no rows were changed.

In [536]:
cursor.rowcount

1

Our cursor object's rowcount returned 1.  This is because the above UPDATE query only impacted 1 row.

In [537]:
pd.read_sql('SELECT id, first, last FROM authors', connection, index_col=['id'])

,first,last
id,,
1,Paul,Deitel
2,Harvey,Deitel
3,Abbey,Deitel
4,Dan,Quirk
5,Alexander,Wald
6,Sue,Black


We can see that Sue Red's name has been changed to Sue Black.  Thanks to the WHERE clause in our UPDATE statement, all other rows have retained their original last name instead of being updated like Sue's was.  It is wise to use a SELECT statement to confirm your query is targeting the correct rows before executing an UPDATE statement.

<hr>

## 7. DELETE FROM

The DELETE FROM statement removes rows from a table.  Let's remove Sue Black to return the authors table to its original condition.  We'll specify Sue's author ID to ensure that she is the only row removed from the authors table.

In [538]:
cursor = cursor.execute('DELETE FROM authors WHERE id=6')

The syntax in the DELETE FROM statement is similar to the INSERT INTO and UPDATE statement.  DELETE FROM selects the table to remove rows.  WHERE filters the removal so that rows are only removed if they meet the criteria of the WHERE clause.

We can verify that the query was run successfully using rowcount once more.

In [539]:
cursor.rowcount

1

A 1 was returned, indicating only one row was removed from the authors table.  Let's check the table to confirm that our desired row was removed.

In [540]:
pd.read_sql('SELECT id, first, last FROM authors', connection, index_col=['id'])

,first,last
id,,
1,Paul,Deitel
2,Harvey,Deitel
3,Abbey,Deitel
4,Dan,Quirk
5,Alexander,Wald


Sue Black has been removed from the authors table as we expected.

<hr>

## 8. Final Results

Let's check on all of our tables using "SELECT *" to return all columns for each table.

In [541]:
pd.read_sql('SELECT * FROM authors', connection)

,id,first,last
0,1,Paul,Deitel
1,2,Harvey,Deitel
2,3,Abbey,Deitel
3,4,Dan,Quirk
4,5,Alexander,Wald


In [542]:
pd.read_sql('SELECT * FROM author_ISBN', connection)

,id,isbn
0,1,0134289366
1,2,0134289366
2,5,0134289366
3,1,0135404673
4,2,0135404673
5,1,0132151006
6,2,0132151006
7,3,0132151006
8,1,0134743350
9,2,0134743350


In [543]:
pd.read_sql('SELECT * FROM titles', connection)

,isbn,title,edition,copyright
0,0135404673,Intro to Python for CS and DS,1,2020
1,0132151006,Internet & WWW How to Program,5,2012
2,0134743350,Java How to Program,11,2018
3,0133976890,C How to Program,8,2016
4,0133406954,Visual Basic 2012 How to Program,6,2014
5,0134601548,Visual C# How to Program,6,2017
6,0136151574,Visual C++ How to Program,2,2008
7,0134448235,C++ How to Program,10,2017
8,0134444302,Android How to Program,3,2017
9,0134289366,Android 6 for Programmers,3,2016
